In [6]:
import sys, os, re; sys.path = (["../src/", "../"] if re.match(r'^(\w\:\\)|(/)', os.getcwd()) else [])+ sys.path 

from typing import Any, Optional, List

import qubx
%qubxd dev 

%load_ext autoreload
%autoreload 2
import asyncio

from qubx import lookup, logger
from qubx.core.strategy import IStrategy, StrategyContext, TriggerEvent
from qubx.pandaz.utils import *

# - - - - - - - - - - - - - - - - - - - - - - - -
from qubx.trackers import PortfolioRebalancerTracker
from qubx.core.series import Quote
from qubx.data.readers import CsvStorageDataReader, AsTimestampedRecords, AsQuotes, RestoreTicksFromOHLC
from qubx.core.basics import Deal, Instrument, Order, Position, Signal
from qubx.core.utils import recognize_time

from qubx.impl.ccxt_connector import CCXTDataConnector
from qubx.impl.ccxt_trading import CCXTSyncTradingConnector
from qubx.utils.runner import get_account_config

def Q(time: str, bid: float, ask: float) -> Quote:
    return Quote(recognize_time(time), bid, ask, 0, 0)

 >  [dev] installing cython rebuilding hook
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
class TestStrategy(IStrategy):
    capital_invested: int = 1000

    def on_start(self, ctx: StrategyContext):
        logger.info(f"Strategy started | {ctx.time()}")

    def on_fit(self, ctx: 'StrategyContext', fit_time: str | pd.Timestamp, previous_fit_time: str | pd.Timestamp | None = None):
        logger.info(f"> Fit is called | fit_time: {fit_time} / prev: {previous_fit_time}")

    def on_event(self, ctx: StrategyContext, event: TriggerEvent) -> List[Signal] | None:
        logger.info(f"{event.time} -> {event}")
        return None

    def ohlcs(self, timeframe:str) -> Dict[str, pd.DataFrame]:
        return {s.symbol: self.ctx.ohlc(s, timeframe).pd() for s in self.ctx.instruments}

    def on_stop(self, ctx: StrategyContext):
        logger.info(f"> test is stopped")

    def tracker(self, ctx: StrategyContext) -> PortfolioRebalancerTracker:
        return PortfolioRebalancerTracker(ctx, 1000, 0)

# Simulated broker and paper trading

In [3]:
from qubx.backtester.simulator import SimulatedExchangeService

In [4]:
auth = get_account_config('binance-qbx', './configs/.env')

sbroker = SimulatedExchangeService('binance', 10000, 'vip0_usdt', 'USDT')
data = CCXTDataConnector('binance', sbroker, **auth)

instruments = [lookup.find_symbol('BINANCE', 'BTCUSDT')]
ctx = StrategyContext(
    TestStrategy(), dict(
        capital_invested=100,
        trading_allowed=False
    ), 
    data, sbroker, instruments=instruments,  # type: ignore
    md_subscription=dict(type='ohlc', timeframe='1Min', nback=0),
    trigger_spec='1Min -5Sec',
)
S = ctx.strategy

2024-07-11 11:23:22.406 [ ℹ️ ] binance initialized - current time 1970-01-01T00:00:00.000000000
2024-07-11 11:23:22.407 [ ⚠️ ] qubx.core.loggers:__init__:324 - Log writer is not defined - strategy activity will not be saved !
2024-07-11 11:23:22.408 [ ℹ️ ] (StrategyContext) set strategy parameters:
	set capital_invested <- 100
2024-07-11 11:23:22.408 [ 🐞 ] Triggering strategy on every 1Min bar after 55S
2024-07-11 11:23:22.407 [ ⚠️ ] qubx.core.loggers:__init__:324 - Log writer is not defined - strategy activity will not be saved !
2024-07-11 11:23:22.408 [ ℹ️ ] (StrategyContext) set strategy parameters:
	set capital_invested <- 100
2024-07-11 11:23:22.408 [ 🐞 ] Triggering strategy on every 1Min bar after 55S


In [5]:
ctx.start()

2024-07-11 11:23:25.109 [ ℹ️ ] (StrategyContext) Subscribing to ohlc updates using {'timeframe': '1Min', 'nback': 0} for 
	['BTCUSDT'] 
2024-07-11 11:23:25.109 [ ℹ️ ] Subscribed on ohlc updates for 1 symbols: 
		['BTCUSDT']
2024-07-11 11:23:25.122 [ ℹ️ ] Strategy started | 1970-01-01T00:00:00.000000000
2024-07-11 11:23:25.123 [ ℹ️ ] (StrategyContext) Start processing market data
2024-07-11 11:23:25.124 [ ℹ️ ] (StrategyContext) strategy is started


2024-07-11 11:23:31.409 [ 🐞 ] [2024-07-11T08:23:31.640442880]: Invoking TestStrategy on_fit('1970-01-01 00:00:00', 'None')
2024-07-11 11:23:31.409 [ ℹ️ ] > Fit is called | fit_time: 1970-01-01 00:00:00 / prev: None
2024-07-11 11:23:31.409 [ 🐞 ] [2024-07-11T08:23:31.640442880]: TestStrategy is fitted
2024-07-11 11:23:55.740 [ ℹ️ ] 2024-07-11T08:23:55.954088960 -> TriggerEvent(time=numpy.datetime64('2024-07-11T08:23:55.954088960'), type='bar', instrument=Instrument(symbol='BTCUSDT', market_type='CRYPTO', exchange='BINANCE', base='BTC', quote='USDT', margin_symbol=None, min_tick=0.01, min_size_step=1e-05, min_size=1e-05, futures_info=None, _aux_instrument=None), data={o:58142.470000 | h:58173.650000 | l:58140.000000 | c:58144.820000 | v:746402.094067})
2024-07-11 11:24:56.053 [ ℹ️ ] 2024-07-11T08:24:56.263703296 -> TriggerEvent(time=numpy.datetime64('2024-07-11T08:24:56.263703296'), type='bar', instrument=Instrument(symbol='BTCUSDT', market_type='CRYPTO', exchange='BINANCE', base='BTC', q

In [31]:
import threading
import time

async def get_bars(symbol):
    ohlcv = await data._exchange.fetch_ohlcv(symbol, '1m')  # type: ignore
    return ohlcv

def run_coro_sync():
    fut = asyncio.run_coroutine_threadsafe(get_bars('BTCUSDT'), data._loop) 
    finished = threading.Event()
    def fut_finished_cb(_):
        print("JJJJJJJJJJJJJ")
        finished.set()
    fut.add_done_callback(fut_finished_cb)
    while not finished.is_set():
        ...
    print("kkkkkkkk")
    return fut.result()

JJJJJJJJJJJJJ
kkkkkkkk


In [ ]:
import threading
import asyncio

def run_coro_threadsafe(coro, other_loop, our_loop = None):
    """Schedules coro in other_loop, awaits until coro has run and returns
    its result.
    """
    loop = our_loop or asyncio.get_event_loop()

    # schedule coro safely in other_loop, get a concurrent.future back
    # NOTE run_coroutine_threadsafe requires Python 3.5.1
    fut = asyncio.run_coroutine_threadsafe(coro, other_loop)

    # set up a threading.Event that fires when the future is finished
    finished = threading.Event()
    def fut_finished_cb(_):
        finished.set()
    fut.add_done_callback(fut_finished_cb)

    # wait on that event in an executor, yielding control to our_loop
    await loop.run_in_executor(None, finished.wait)

    # coro's result is now available in the future object
    return fut.result()

In [24]:
# fut = data._loop.run_until_complete(get_bars('BTCUSDT'))

RuntimeError: This event loop is already running

In [22]:
d.result()

<coroutine object get_bars at 0x0000022F3118A880>

In [137]:
print(sbroker.get_position(instruments[0]))
print(sbroker.get_orders())

2024-07-11 05:56:34 [BINANCE:BTCUSDT]       0.00000      0.00    -0.6950 | 58009.35        0.00
[]


In [101]:
ctx.trade('BTCUSDT', 0.1)

2024-07-11 08:55:12.110 [ ℹ️ ] (StrategyContext) sending market buy for 0.1 of BTCUSDT ...
2024-07-11 08:55:12.110 [ 🐞 ] [OMS] BTCUSDT - SIM-ORDER-BTCUSDT-100002 MARKET BUY 0.1 executed at 57986.005
2024-07-11 08:55:12.110 [ ℹ️ ]   ::  traded 0.1 for BTCUSDT @ 57986.005 -> 0.00
2024-07-11 08:55:12.110 [ ℹ️ ] Order SIM-ORDER-BTCUSDT-100002 MARKET BUY 0.1 of BTCUSDT -> CLOSED


Order(id='SIM-ORDER-BTCUSDT-100002', type='MARKET', symbol='BTCUSDT', time=numpy.datetime64('2024-07-11T05:55:11.701357312'), quantity=0.1, price=0, side='BUY', status='CLOSED', time_in_force='gtc', client_id='TestStrategy_BTCUSDT_2', cost=0.0)

In [122]:
ctx.trade('BTCUSDT', -0.1)

2024-07-11 08:55:28.803 [ ℹ️ ] (StrategyContext) sending market sell for 0.1 of BTCUSDT ...
2024-07-11 08:55:28.804 [ 🐞 ] [OMS] BTCUSDT - SIM-ORDER-BTCUSDT-100004 MARKET SELL 0.1 executed at 57978.015
2024-07-11 08:55:28.805 [ ℹ️ ]   ::  traded -0.1 for BTCUSDT @ 57978.015 -> -0.35
2024-07-11 08:55:28.805 [ ℹ️ ] Order SIM-ORDER-BTCUSDT-100004 MARKET SELL 0.1 of BTCUSDT -> CLOSED


Order(id='SIM-ORDER-BTCUSDT-100004', type='MARKET', symbol='BTCUSDT', time=numpy.datetime64('2024-07-11T05:55:28.608849920'), quantity=0.1, price=0, side='SELL', status='CLOSED', time_in_force='gtc', client_id='TestStrategy_BTCUSDT_4', cost=0.0)

In [124]:
ctx.trade('BTCUSDT', 0.1, 45000)

2024-07-11 08:55:43.969 [ ℹ️ ] (StrategyContext) sending limit buy for 0.1 of BTCUSDT ...
2024-07-11 08:55:43.969 [ 🐞 ] [OMS] BTCUSDT - registered SIM-ORDER-BTCUSDT-100005 LIMIT BUY 0.1 45000


Order(id='SIM-ORDER-BTCUSDT-100005', type='LIMIT', symbol='BTCUSDT', time=numpy.datetime64('2024-07-11T05:55:41.389935104'), quantity=0.1, price=45000, side='BUY', status='OPEN', time_in_force='gtc', client_id='TestStrategy_BTCUSDT_5', cost=0.0)

In [136]:
ctx.cancel('BTCUSDT')

2024-07-11 08:56:31.011 [ 🐞 ] [OMS] BTCUSDT - SIM-ORDER-BTCUSDT-100005 LIMIT BUY 0.1 canceled
2024-07-11 08:56:31.011 [ ℹ️ ] Order SIM-ORDER-BTCUSDT-100005 LIMIT BUY 0.1 of BTCUSDT -> CANCELED
2024-07-11 08:56:31.011 [ ℹ️ ] Order SIM-ORDER-BTCUSDT-100005 LIMIT BUY 0.1 of BTCUSDT -> CANCELED


In [138]:
ctx.stop()
data.close()

2024-07-11 08:56:41.983 [ ℹ️ ] > test is stopped
2024-07-11 08:56:41.984 [ ℹ️ ] 	CachedMarketDataHolder.update_by_bar took 0.0000314 secs
2024-07-11 08:56:41.984 [ ℹ️ ] 	StrategyContext._processing_bar took 0.0000478 secs
2024-07-11 08:56:41.984 [ ℹ️ ] 	StrategyContext.handler took 0.0000026 secs
2024-07-11 08:56:41.984 [ ℹ️ ] 	StrategyContext._process_incoming_data took 0.9502193 secs
2024-07-11 08:56:41.984 [ ℹ️ ] 	loggers.notify took 0.0000022 secs
2024-07-11 08:56:41.984 [ ℹ️ ] 	strategy.on_fit took 0.0018189 secs
2024-07-11 08:56:41.984 [ ℹ️ ] 	strategy.on_event took 0.0013189 secs
2024-07-11 08:56:41.984 [ ℹ️ ] 	StrategyContext.trade took 0.0014009 secs
2024-07-11 08:56:41.984 [ ℹ️ ] 	StrategyContext._processing_order took 0.0006980 secs
2024-07-11 08:56:41.984 [ ℹ️ ] 	StrategyContext._processing_deals took 0.0005858 secs
2024-07-11 08:56:41.984 [ ℹ️ ] 	StrategyContext.cancel took 0.0014524 secs
2024-07-11 08:56:41.984 [ ❌ ] This event loop is already running


c:\Users\dmitr\devs\Qubx\experiments\../src\qubx\impl\ccxt_connector.py:285: RuntimeWarning: coroutine 'Exchange.close' was never awaited
  logger.error(e)


# Simulated data provider

In [2]:
r = CsvStorageDataReader('../tests/data/csv')

In [3]:
instr = lookup.find_symbol("BINANCE.UM", "BTCUSDT")
stream = r.read('BTCUSDT_ohlcv_M1', transform=RestoreTicksFromOHLC(trades=False, spread=instr.min_tick))

In [ ]:
from qubx.backtester.simulator import SimulatedExchangeService